<a href="https://colab.research.google.com/github/kmk4444/Knowledge-graph-for-rag/blob/main/Query_with_cypher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 2: Querying Knowledge Graphs with Cypher

### Import packages and set up Neo4

In [9]:
!pip install langchain-community neo4j

In [5]:
#from dotenv import load_dotenv


from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [14]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

### Querying the movie knowledge graph
- Match all nodes in the graph

In [15]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """

In [16]:
result = kg.query(cypher)
result

[{'count(n)': 105}]

In [17]:
cypher = """
  MATCH (n)
  RETURN count(n) AS numberOfNodes
  """

In [18]:
result = kg.query(cypher)
result

[{'numberOfNodes': 105}]

In [19]:
print(f"There are {result[0]['numberOfNodes']} nodes in this graph.")

There are 105 nodes in this graph.


- Match only the `Movie` nodes by specifying the node label

In [21]:
cypher = """
  MATCH (n:Movie)
  RETURN count(n) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 21}]

- Change the variable name in the node pattern match for improved readability

In [22]:
cypher = """
  MATCH (m:Movie)
  RETURN count(m) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 21}]

- Match only the `Person` nodes

In [23]:
cypher = """
  MATCH (p:Person)
  RETURN count(p) AS numberOfPeople
  """
kg.query(cypher)

[{'numberOfPeople': 84}]

- Match a single person by specifying the value of the `name` property on the `Person` node

In [24]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})
  RETURN tom
  """
kg.query(cypher)

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

- Match a single `Movie` by specifying the value of the `title` property

In [26]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"})
  RETURN cloudAtlas
  """
kg.query(cypher)

[]

In [41]:
cypher = """
  MATCH (P:Person)
  RETURN P LIMIT 5
  """
kg.query(cypher)

[{'P': {'born': 1964, 'name': 'Keanu Reeves'}},
 {'P': {'born': 1967, 'name': 'Carrie-Anne Moss'}},
 {'P': {'born': 1961, 'name': 'Laurence Fishburne'}},
 {'P': {'born': 1960, 'name': 'Hugo Weaving'}},
 {'P': {'born': 1967, 'name': 'Lilly Wachowski'}}]

In [38]:
cypher = """
  MATCH (M:Movie)
  RETURN M LIMIT 5
  """
kg.query(cypher)

[{'M': {'tagline': 'Welcome to the Real World',
   'title': 'The Matrix',
   'released': 1999}},
 {'M': {'tagline': 'Free your mind',
   'title': 'The Matrix Reloaded',
   'released': 2003}},
 {'M': {'tagline': 'Everything that has a beginning has an end',
   'title': 'The Matrix Revolutions',
   'released': 2003}},
 {'M': {'tagline': 'Evil has its winning ways',
   'title': "The Devil's Advocate",
   'released': 1997}},
 {'M': {'tagline': "In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.",
   'title': 'A Few Good Men',
   'released': 1992}}]

- Return only the `released` property of the matched `Movie` node

In [30]:
cypher = """
  MATCH (StandByMe:Movie {title:"Stand By Me"})
  RETURN StandByMe.released
  """
kg.query(cypher)

[{'StandByMe.released': 1986}]

- Return two properties

In [31]:
cypher = """
  MATCH (StandByMe:Movie {title:"Stand By Me"})
  RETURN StandByMe.released, StandByMe.tagline
  """
kg.query(cypher)

[{'StandByMe.released': 1986,
  'StandByMe.tagline': "For some, it's the last real taste of innocence, and the first real taste of life. But for everyone, it's the time that memories are made of."}]

### Cypher patterns with conditional matching

In [32]:
cypher = """
  MATCH (nineties:Movie)
  WHERE nineties.released >= 1900
    AND nineties.released < 2000
  RETURN nineties.title
  """
kg.query(cypher)

[{'nineties.title': 'Top Gun'},
 {'nineties.title': 'Stand By Me'},
 {'nineties.title': 'Joe Versus the Volcano'},
 {'nineties.title': 'A Few Good Men'},
 {'nineties.title': 'Unforgiven'},
 {'nineties.title': 'Sleepless in Seattle'},
 {'nineties.title': 'Johnny Mnemonic'},
 {'nineties.title': 'That Thing You Do'},
 {'nineties.title': 'The Birdcage'},
 {'nineties.title': "The Devil's Advocate"},
 {'nineties.title': 'As Good as It Gets'},
 {'nineties.title': 'What Dreams May Come'},
 {'nineties.title': "You've Got Mail"},
 {'nineties.title': 'When Harry Met Sally'},
 {'nineties.title': 'The Matrix'},
 {'nineties.title': 'Snow Falling on Cedars'}]

### Pattern matching with multiple nodes

In [34]:
cypher = """
  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
  RETURN actor.name, movie.title LIMIT 10
  """
kg.query(cypher)

[{'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix'},
 {'actor.name': 'Emil Eifrem', 'movie.title': 'The Matrix'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Revolutions'}]

In [35]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie)
  RETURN tom.name, tomHanksMovies.title
  """
kg.query(cypher)

[{'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "You've Got Mail"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Sleepless in Seattle'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Joe Versus the Volcano'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'That Thing You Do'}]

In [43]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors)
  RETURN coActors.name, m.title
  """
kg.query(cypher)

[{'coActors.name': 'Meg Ryan', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Greg Kinnear', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Parker Posey', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Dave Chappelle', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Steve Zahn', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Meg Ryan', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Rita Wilson', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Bill Pullman', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Victor Garber', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': "Rosie O'Donnell", 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Meg Ryan', 'm.title': 'Joe Versus the Volcano'},
 {'coActors.name': 'Nathan Lane', 'm.title': 'Joe Versus the Volcano'},
 {'coActors.name': 'Charlize Theron', 'm.title': 'That Thing You Do'},
 {'coActors.name': 'Liv Tyler', 'm.title': 'That Thing You Do'}]